In [1]:
import os
import sys
import numpy as np
import torch
from torch import nn
from transformers import pytorch_utils as torch_utils
from peft import LoraConfig

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu'
device

'mps'

In [3]:
import importlib
import src.train
import src.model

importlib.reload(src.train)
importlib.reload(src.model)

from src.train import sft_train_lora
from src.model import identify_target_modules
from data.sudoku import Sudoku

In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset

In [5]:
%pip install ipywidgets

Note: you may need to restart the kernel to use updated packages.


In [6]:
from huggingface_hub import login
login()

In [ ]:
# dataset = load_dataset("lucasmccabe-lmi/CodeAlpaca-20k", split="train")


model = AutoModelForCausalLM.from_pretrained("facebook/opt-350m")
tokenizer = AutoTokenizer.from_pretrained("facebook/opt-350m")



# insert your path to sudoku dataset here
dataset = Sudoku(data_file="",tokenizer=tokenizer)

In [13]:
dataset[0]

{'input_ids': tensor([    2, 18377,     5, 12314, 20714, 18108,   112,  7586,   245,     4,
          3272,  7586,   401,     4,   246,  7586,   398,     4,   466, 43796,
          5208,   734,   134, 44629,   398,  5067,   134, 42514,  7586,   401,
         42514,   734,   406,     4,   398,     4,   466,     4,  5067,     4,
          3706,   734,   401,     4, 29930,     6,    61,    34,   974, 14885,
             8,    10,  9600,   691,     9,   132,     4,   176,     4,  3401,
          6136,    13,     5,   507,  9804,     4, 22560, 31652,    35, 30858,
          4283,  3272,  2481, 37853, 30442,  4156,   996, 31782, 42080,  3416,
         26629,  2022,  2481,  4652,  5067,  1646,  1978,  2022, 23821, 37202,
         26340,  3761, 23821, 43587,  3170,  1646,  2518,  3506, 29156,  5208,
          4027, 29930]),
 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

In [14]:
target_modules = identify_target_modules(model, name_segment='self_attn')
print(target_modules)

In [15]:
lora_config = LoraConfig(
    target_modules=target_modules,
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
)

In [16]:
sft_train_lora(
    base_model=model,
    train_dataset=dataset,
    eval_dataset=dataset,
    tokenizer=AutoTokenizer.from_pretrained("facebook/opt-350m"),
    adapter_name="sft_lora",
    response_template=" ### Answer:",
    lora_config=lora_config,
)

/opt/anaconda3/envs/reasoning-decomp/lib/python3.12/site-packages/trl/trainer/sft_trainer.py:292: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


  0%|          | 0/1125000 [00:00<?, ?it/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


KeyboardInterrupt: 